In [32]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')


from data_downloader import DataDownloader
from var_model import VAR
from dice_model import DiceModel

In [33]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA', 'NEE', 'FSLR', 'VWDRY', 'BEP'] 
benchmark = '^GSPC'  
start_date = '2014-12-01'
end_date = '2024-05-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date=start_date, end_date=end_date,
                                                                      assets=assets, benchmark=benchmark)

[*********************100%%**********************]  9 of 9 completed
[*********************100%%**********************]  1 of 1 completed


In [34]:
asset_prices.head()

,AAPL,BEP,FSLR,GOOG,IBM,NEE,NVDA,TSLA,VWDRY
Date,,,,,,,,,
2014-12-01,25.653625,10.002731,45.759998,26.551716,101.126076,20.429741,0.493978,15.442667,2.201333
2014-12-02,25.555540,10.080499,46.419998,26.549229,101.833496,20.488214,0.494698,15.428667,2.266397
2014-12-03,25.845356,10.216592,47.119999,26.428360,102.991585,20.347870,0.507420,15.286667,2.259167
2014-12-04,25.747267,10.167986,46.720001,26.726309,102.697395,20.172440,0.502859,15.218667,2.289892
2014-12-05,25.638023,9.908764,46.910000,26.126928,102.209068,20.063280,0.505739,14.914000,2.371222


In [35]:
# Cargar archivos CSV
temperature = pd.read_csv('Data/Temperature.csv')
drought = pd.read_csv('Data/Drought_Severity.csv')

# Cargar archivo XLSX
co2_emission = pd.read_excel('Data/CO2_Emissions.xlsx', skiprows=10).drop(index=0)
co2_emission = co2_emission[["Month",
                             "Coal, Including Coal Coke Net Imports, CO2 Emissions",
                             "Natural Gas, Excluding Supplemental Gaseous Fuels, CO2 Emissions", 
                             "Petroleum, Excluding Biofuels, CO2 Emissions", 
                             "Total Energy CO2 Emissions"]]
co2_emission = co2_emission.rename(columns={
    "Month": "Date",
    "Coal, Including Coal Coke Net Imports, CO2 Emissions": "Coal",
    "Natural Gas, Excluding Supplemental Gaseous Fuels, CO2 Emissions": "Natural Gas",
    "Petroleum, Excluding Biofuels, CO2 Emissions": "Petroleum",
    "Total Energy CO2 Emissions": "Total CO2 Emissions"
})

# Convertir la columna 'Date' a formato datetime e indexarla
co2_emission['Date'] = pd.to_datetime(co2_emission['Date'])

# Aplicar el porcentaje de cambio interanual (year-to-year) para cada serie de CO2 (sin incluir la columna 'Date')
co2_emission_pct_change = co2_emission.set_index('Date').pct_change(periods=12)

# Eliminar los valores nulos generados por el cálculo del porcentaje de cambio
co2_emission_pct_change = co2_emission_pct_change.dropna()

# Restablecer el índice para que 'Date' vuelva a ser una columna
co2_emission_pct_change = co2_emission_pct_change.reset_index()

# Asignar el resultado final
co2_emission = co2_emission_pct_change

co2_emission


,Date,Coal,Natural Gas,Petroleum,Total CO2 Emissions
0,1974-01-01,-0.010579,-0.046322,-0.081750,-0.054783
1,1974-02-01,-0.008044,-0.032009,-0.098440,-0.058713
2,1974-03-01,0.010560,-0.003760,-0.076896,-0.036424
3,1974-04-01,0.003739,-0.074031,-0.001288,-0.019331
4,1974-05-01,0.024103,-0.066917,-0.063255,-0.041983
...,...,...,...,...,...
600,2024-01-01,0.167510,0.120100,0.010273,0.081480
601,2024-02-01,-0.041299,0.006871,0.007048,0.000088
602,2024-03-01,-0.210958,-0.074042,-0.042391,-0.078618
603,2024-04-01,-0.072617,-0.010663,0.001523,-0.012618


In [36]:
# Function to drop one or multiple columns from a dataframe
def drop_columns(df, columns):
    df = df.drop(columns=columns)   
    return df

# Function to rename columns in a dataframe
def rename_columns(df, columns):
    df = df.rename(columns=columns)
    return df

# Drop the 'Average surface temperature.1', 'Code', 'Entity', 'year' columns from the temperature dataframe
temperature = drop_columns(temperature, ['Average surface temperature.1','Code', 'Entity', 'year'])

# Rename the columns in the temperature dataframe 
temperature = rename_columns(temperature, {'Day': 'Date',
                                           'Average surface temperature': 'Temperature'})
temperature['Date'] = pd.to_datetime(temperature['Date'], format='%d/%m/%y', errors='coerce')
temperature['Date'] = temperature['Date'].apply(lambda x: x.replace(year=x.year - 100) if x.year >= 2025 else x)
temperature = temperature[(temperature['Date'] >= '1940-01-01') & (temperature['Date'] <= '2024-12-31')]
temperature['Date'] = temperature['Date'].apply(lambda x: x.replace(day=1))

# Excluir la columna 'Date' para aplicar pct_change solo a las columnas numéricas
temperature_pct = temperature.drop(columns=['Date']).pct_change(periods=12)

# Volver a agregar la columna 'Date' al dataframe resultante
temperature_pct['Date'] = temperature['Date']

# Eliminar los valores nulos generados por el cálculo del porcentaje de cambio
temperature_pct = temperature_pct.dropna()

# Reordenar las columnas para que 'Date' esté al principio
cols = ['Date'] + [col for col in temperature_pct.columns if col != 'Date']
temperature = temperature_pct[cols]

temperature


,Date,Temperature
12,1941-01-01,-0.378235
13,1941-02-01,-0.183787
14,1941-03-01,-2.615123
15,1941-04-01,0.002567
16,1941-05-01,-0.012651
...,...,...
1011,2024-04-01,0.234115
1012,2024-05-01,-0.026968
1013,2024-06-01,0.072609
1014,2024-07-01,-0.026005


In [37]:
# Eliminar columnas innecesarias
if 'Code' in drought.columns:
    drought = drought.drop(columns=['Code'])
if 'Name' in drought.columns:
    drought = drought.drop(columns=['Name'])

drought = rename_columns(drought, {'MapDate': 'Date'})
drought['Date'] = pd.to_datetime(drought['Date'], format='%Y%m%d')

# Extraer el año y mes
drought['Year'] = drought['Date'].dt.year
drought['Month'] = drought['Date'].dt.month

# Asegurarte de que la columna 'DSCI' sea numérica y eliminar NaNs
drought['DSCI'] = pd.to_numeric(drought['DSCI'], errors='coerce')

# Agrupar por año y mes y calcular el promedio
monthly_avg = drought.groupby(['Year', 'Month'])['DSCI'].mean().reset_index()
monthly_avg['Month'] = monthly_avg['Month'].fillna(0).astype(int)

# Formatear la fecha como aaaa/mm/01
monthly_avg['Date'] = monthly_avg.apply(lambda row: f"{int(row['Year'])}-{int(row['Month']):02d}-01", axis=1)
drought['Date'] = pd.to_datetime(drought['Date'], format='%Y%m%d')
drought = monthly_avg[['Date', 'DSCI']]

# Excluir la columna 'Date' para aplicar pct_change solo a las columnas numéricas
drought_pct = drought.drop(columns=['Date']).pct_change(periods=12)

# Volver a agregar la columna 'Date' al dataframe resultante
drought_pct['Date'] = drought['Date']

# Eliminar los valores nulos generados por el cálculo del porcentaje de cambio
drought_pct = drought_pct.dropna()


drought

,Date,DSCI
0,2014-12-01,104.000000
1,2015-01-01,103.250000
2,2015-02-01,111.250000
3,2015-03-01,117.200000
4,2015-04-01,125.250000
...,...,...
113,2024-05-01,49.000000
114,2024-06-01,51.500000
115,2024-07-01,75.000000
116,2024-08-01,89.250000


In [38]:
# Asegurarse de que la columna 'Date' en todos los dataframes esté en formato datetime
temperature['Date'] = pd.to_datetime(temperature['Date'], errors='coerce')
drought['Date'] = pd.to_datetime(drought['Date'], errors='coerce')

# Verificar si la columna 'Date' existe en el dataframe co2_emission
if 'Date' in co2_emission.columns:
    co2_emission['Date'] = pd.to_datetime(co2_emission['Date'], errors='coerce')
else:
    # Identificar si hay otra columna que debería ser la 'Date'
    potential_date_column = [col for col in co2_emission.columns if 'date' in col.lower()]
    if potential_date_column:
        co2_emission = co2_emission.rename(columns={potential_date_column[0]: 'Date'})
        co2_emission['Date'] = pd.to_datetime(co2_emission['Date'], errors='coerce')
    else:
        raise KeyError("The 'Date' column is missing in the co2_emission dataframe")

# Realizar la fusión de los dataframes en la columna 'Date'
data = pd.merge(temperature, drought, on='Date', how='inner')
data = pd.merge(data, co2_emission, on='Date', how='inner')

data


,Date,Temperature,DSCI,Coal,Natural Gas,Petroleum,Total CO2 Emissions
0,2014-12-01,-1.134168,104.00,-0.107713,-0.079697,0.035305,-0.045066
1,2015-01-01,-0.100921,103.25,-0.140308,-0.024601,0.005973,-0.049579
2,2015-02-01,-0.348977,111.25,-0.117793,0.071572,0.034858,-0.002740
3,2015-03-01,0.936614,117.20,-0.185240,0.017076,0.057500,-0.029565
4,2015-04-01,0.092542,125.25,-0.163193,0.027977,0.012721,-0.034143
...,...,...,...,...,...,...,...
109,2024-01-01,4.009622,98.80,0.167510,0.120100,0.010273,0.081480
110,2024-02-01,-5.464459,72.75,-0.041299,0.006871,0.007048,0.000088
111,2024-03-01,0.806495,74.25,-0.210958,-0.074042,-0.042391,-0.078618
112,2024-04-01,0.234115,63.00,-0.072617,-0.010663,0.001523,-0.012618


In [39]:
# Function to calculate the monthly returns of a dataframe
def calculate_monthly_returns(df):
    if 'Date' in df.columns:
        df = df.set_index('Date')
    
    # Resample to monthly frequency and get the last value of each month
    df = df.resample('M').last()
    
    # Calculate percentage change (returns) and drop NaN values
    df = df.pct_change().dropna()
    
    return df

# Example returns calculation
asset_returns = calculate_monthly_returns(asset_prices)
benchmark_returns = calculate_monthly_returns(benchmark_prices)

# Reset index to turn the Date back into a column
asset_returns = asset_returns.reset_index()

# Modify the Date column to set the day to the 1st of each month
asset_returns['Date'] = asset_returns['Date'].apply(lambda x: x.replace(day=1))

# Display the first rows of the asset returns
asset_returns.head()


,Date,AAPL,BEP,FSLR,GOOG,IBM,NEE,NVDA,TSLA,VWDRY
0,2015-01-01,0.061424,0.039767,-0.051121,0.015426,-0.044440,0.027754,-0.042394,-0.084574,0.078838
1,2015-02-01,0.100777,-0.030332,0.411862,0.044676,0.063701,-0.045986,0.153382,-0.001277,0.076154
2,2015-03-01,-0.031372,0.026324,0.000669,-0.018625,-0.008892,0.005703,-0.051224,-0.071653,-0.007800
3,2015-04-01,0.005787,0.032933,-0.002007,-0.016760,0.067228,-0.029985,0.060679,0.197489,0.094161
4,2015-05-01,0.045339,-0.058228,-0.166918,-0.009733,-0.002079,0.021757,0.001461,0.109489,0.134089


In [40]:
data_pct = calculate_monthly_returns(data)
data_pct


,Temperature,DSCI,Coal,Natural Gas,Petroleum,Total CO2 Emissions
Date,,,,,,
2015-01-31,-0.911018,-0.007212,0.302614,-0.691324,-0.830817,0.100132
2015-02-28,2.457922,0.077482,-0.160469,-3.909345,4.835952,-0.944744
2015-03-31,-3.683888,0.053483,0.572591,-0.761415,0.649566,9.791847
2015-04-30,-0.901195,0.068686,-0.119021,0.638389,-0.778773,0.154872
2015-05-31,-0.938632,-0.095808,-0.322127,0.099687,1.009213,-0.542862
...,...,...,...,...,...,...
2024-01-31,-3.360192,-0.137118,-1.742080,-2.933178,1.346562,-2.263747
2024-02-29,-2.362836,-0.263664,-1.246548,-0.942786,-0.313946,-0.998924
2024-03-31,-1.147589,0.020619,4.108020,-11.775308,-7.014563,-897.645224


In [ ]:
asset_returns.plot(x='Date', y=assets, figsize=(14, 7), title='Asset Returns')